# Earth System Model Data Analysis with Python and Xarray

*Paolo Davini (2024)*

This notebook aims at illustrating how to perform some simple climate data analysis based on the [NetCDF](https://www.unidata.ucar.edu/software/netcdf/) file format, which are geo-referenced files common in climate modeling. All the analysis is done with Python3, taking advantages of powerfule package known as [Xarray](https://docs.xarray.dev/en/stable/). The idea is to show some basic functionalities to access data from climate models, select some years, doing averages and so on. For more advanced usage, check the [Xarray tutorials](https://tutorial.xarray.dev/intro.html) or simply ask ChatGPT.

We start by loading fundamental Python packages. In Python, imports can be done only for packages already available on the machines - the required ones have been installed in this virtual machine - and a common way to perform imports is to use the *as* syntax which simplifies the call of the corresponding package.

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

Now we will load some data surface temperature data from a climate experiment from CMIP6, specifically for EC-Earth3 historical simulations. `tas` is the *cmor* (a [convention](https://wcrp-cmip.github.io/WGCM_Infrastructure_Panel//cmor_and_mip_tables.html) used in NetCDF climate model data) standard name for 2-meter surface temperature. To simplify the analysis, the data have been interpolated on a regular grid of 1x1 degree. All the pre-processing operations have been with [CDO](https://code.mpimet.mpg.de/projects/cdo), a powerful command line tool developed by MPI.

## NetCDF files and Xarray objects

We start loading the data, using `xr.open_dataset()` method. Very Easy!

In [ ]:
data = xr.open_dataset('ussp/tas_EC-Earth3_historical_r1p1if1_1850-2014.nc')
data

Xarray provides a packed object which includes multiple array, mostly with numpy but also possibly with other formats, all together in specific geo-referenced gridded shape. It is possible to inspect the different properties of the data we just loaded. This is made by `time`, `lon`, `lat` coordinates (which are distinguished from Xarray dimensions, which can be interpreted mostly as the indices of the data). In this case we have a single height level ince we are the surface. Two variables are stored, both `tas` and `time_bnds`. We can further access each variable. Access can be done with both square brackets or dots.

In [ ]:
data['tas']

This is the `DataArray`, typical object and stores a lot of information about the properties of the data (what we call *attributes*). An Xarray `Dataset` is made by multiple `DataArray`. This two object make the fundamental bricks of Xarray. What about the other variable?

In [ ]:
data.time_bnds

So what is `time_bnds` for? What is inside this object and what is telling us? You can inspect the Xarray object using the `.data` attribute

In [ ]:
data.time_bnds.data

## Playing with the data

Xarray is very powerful, and can combine multiple operation in a single line. For example is possible to easily select a subset of the data, operating only on surface temperature, doing the average and then plotting even with a one-liner!

In [ ]:
wrong_mean = data['tas'].mean(dim=['lon', 'lat'])


One important feature of Xarray is that operation are done in **lazy** way. This means that the computation are not executed immediately, but only when they are effectively needed. For example, when you ask for plotting

In [ ]:
wrong_mean.plot()

What are we looking to? **What's this noise?**

We can look closer, perhaps we can get more insight...

In [ ]:
wrong_mean.sel(time=slice('2000-01-01', '2010-01-01')).plot()

What are those cycles? We might need to do an average in time thus, and we can do it with the `resample()` method, to produce a yearly mean!

In [ ]:
wrong_mean = data['tas'].mean(dim=['lon', 'lat']).resample(time='Y').mean()
wrong_mean.plot()

`resample()` is very powerful, build on [pandas](https://pandas.pydata.org/docs/user_guide/timeseries.html) and can resample on almost all the kind of frequency you can imagine.
Of course, we can inspect the seasonal cycle in different part of the globe, and this can be easily done with few lines of code.

In [ ]:
select = data['tas'].sel(time=slice('2000-01-01', '2010-01-01'))
zonal = select.mean(dim='lon').sel(lat=[-89.5, 0.5, 89.5])
zonal.plot.line(x="time")

## A correct spatial averaging

However, most of the above operations are wrong. We did a serious mistake, we did not consider the fact that into a regularly gridded dataset over the globe, the area of each grid point is not the same. Therefore, if we do the average as above, we are counting points at high latitudes with the same weight that points at the Equator, and this is wrong.
Keep in mind that the half of Earth surface is between 30S and 30N!
However, this can be easily achieved with Xarray through the `weighted()` method, keepining mind that the surface of each grid box scales with the cosine of the latitude

In [ ]:
area_weights = np.cos(np.deg2rad(data.lat))

In [ ]:
weighted_mean = data['tas'].weighted(area_weights).mean(dim=['lon', 'lat']).resample(time='Y').mean()

In [ ]:
weighted_mean.plot(label="weighted")
wrong_mean.plot(label="unweighted")
plt.legend()

## A comparison with observations

This how a climate model behaves, but what about real world observations? Gridded datasets are available, but for simplicity we will make use of the ERA5 ECMWF reanalysis, which provides a model-uniformed approach to climate data. Reanalysis are made with high resolution climate models which assimilates observations from all over the world and fills missing points with climate data. They are helpful since they can measure also things which are not usually available on gridded datasets, as snowfall or heat fluxes. 

In [ ]:
era5 = xr.open_mfdataset('ussp/tas_ERA5_1940-2022.nc')
era5

In [ ]:
era5_mean = era5['tas'].weighted(area_weights).mean(dim=['lon', 'lat']).resample(time='YS').mean()

In [ ]:
weighted_mean.plot(label="EC-Earth3", color='green')
era5_mean.plot(label="ERA5", color='purple')
plt.legend()

The model seems to be more sensitive that the real-world. Indeed, EC-Earth3 is known as being one climate model with a large climate sensitivity!

- Why model and reanalysis have a so large difference?
- Can you do the same plot but for the Northern Hemisphere and the Southern Hemisphere only?

## Climatological mean and interannual variability

We can explore the climatological mean of temperature, and also inspect its variability, few simple commands

In [ ]:
clim = data['tas'].sel(time=slice('1980-01-01', '2009-12-31')).mean(dim='time')
clim.plot()

In [ ]:
std = data['tas'].sel(time=slice('1980-01-01', '2009-12-31')).std(dim='time')
std.plot()

Which patterns emerge so clearly from those two maps above?
Why are difference so large in stadard deviation between northern hemisphere and southern hemisphere?

## Time selection and seasonal mean
One thing that is important to be able to do is to select specific season in order to do analysis of some selected data only

In [ ]:
jan = data['tas'].sel(time=(data.time.dt.month==1)).sel(time=slice('2000-01-01', '2009-12-31')).mean(dim='time')
jan.plot()

Let's do the same thing for July, and the run the difference

In [ ]:
jul = data['tas'].sel(time=(data.time.dt.month==7)).sel(time=slice('2000-01-01', '2009-12-31')).mean(dim='time')
delta = (jul - jan).plot()

Which pattern are emerging clearly in this map? Could you plot **the same thing but for Mar and September**? Could you do the difference between the two months? What is emerging in this plot? 

## Plotting with Cartopy 

An important (unfortunately far from perfect) tool in Python is `cartopy`, a package can allow to do different things from adding coastlines or plotting with specific projection.

In [ ]:
import cartopy.crs as ccrs
plotproj = dict(projection=ccrs.Orthographic(0, 35), facecolor="gray")
climate = data['tas'].sel(time=slice('2000-01-01', '2010-01-01')).mean(dim='time')
pp = climate.plot(subplot_kws=plotproj, transform=ccrs.PlateCarree())
pp.axes.coastlines()

In [ ]:
pp = climate.plot.contourf(subplot_kws=dict(projection=ccrs.PlateCarree()))
pp.axes.coastlines()

## A regional analysis
Finally, a simple test to show how to select a box and plot the characteristics of temperature in Italy

In [ ]:
italy = data['tas'].sel(lon=slice(5,20),lat=slice(35,50)).mean(dim='time')
pp = italy.plot(subplot_kws=dict(projection=ccrs.PlateCarree()))
pp.axes.coastlines()

Climate models have a **limited resolution**, and although it is increasing, the entire Italy is made by a few pixels!
- Can you do the same plot over India?
- Can you plot the temperature timeseries for Urbino forecasted by the EC-Earth3 climate model and compare it with ERA5? 